In [1]:
import os
import numpy as np
import json
from PIL import Image

In [2]:
# 경로 설정
DATA_DIR = '/workspace/data/Data_unzipped'
TRAIN_IMG_PATH = os.path.join(DATA_DIR, 'img', 'train')
VALID_IMG_PATH = os.path.join(DATA_DIR, 'img', 'val')
TRAIN_LABEL_PATH = os.path.join(DATA_DIR, 'label', 'train')
VALID_LABEL_PATH = os.path.join(DATA_DIR, 'label', 'val')

TEST_DIR = '/workspace/data/test_unzipped'
TEST_IMG_PATH = os.path.join(TEST_DIR, 'image')
TEST_LABEL_PATH = os.path.join(TEST_DIR, 'label')

IMG_SIZE = (224, 224)

SAVE_DIR = '/workspace/yonghak'
os.makedirs(SAVE_DIR, exist_ok=True)

##### emotion

In [3]:
# 감정 저장
def load_labels(label_path, dataset_type=''):
    if dataset_type:
        label_path = os.path.join(label_path, dataset_type)
    labels = {}
    for file_name in os.listdir(label_path):
        if file_name.endswith('.json'):
            with open(os.path.join(label_path, file_name), 'r', encoding='euc-kr') as f:
                data = json.load(f)
                for item in data:
                    filename = item['filename']
                    face_exp = item['faceExp_uploader']
                    labels[filename] = face_exp
    return labels

In [4]:
def prepare_and_save_all_dataset(
    train_img_dir, train_labels,
    val_img_dir, val_labels,
    test_img_dir, test_labels,
    img_size, save_path
):
    all_images = []
    all_emotions = []
    all_sets = []

    def add_images(img_dir, labels, set_name):
        for root, _, files in os.walk(img_dir):
            for file in files:
                if file in labels:
                    img_path = os.path.join(root, file)
                    image = Image.open(img_path).convert('RGB')
                    image = image.resize(img_size)
                    all_images.append(np.array(image))
                    all_emotions.append(labels[file])
                    all_sets.append(set_name)

    add_images(train_img_dir, train_labels, 'train')
    add_images(val_img_dir, val_labels, 'val')
    add_images(test_img_dir, test_labels, 'test')

    all_images = np.array(all_images)
    all_emotions = np.array(all_emotions)
    all_sets = np.array(all_sets)
    np.savez(save_path, images=all_images, emotions=all_emotions, sets=all_sets)
    print(f"{save_path} 저장 완료 (총 {all_images.shape[0]}장)")

In [5]:
# 실행
save_path_emotion = os.path.join(SAVE_DIR, 'image_emotion_dataset.npz')
train_labels = load_labels(TRAIN_LABEL_PATH)
val_labels = load_labels(VALID_LABEL_PATH)
test_labels = load_labels(TEST_LABEL_PATH)

prepare_and_save_all_dataset(
    TRAIN_IMG_PATH, train_labels,
    VALID_IMG_PATH, val_labels,
    TEST_IMG_PATH, test_labels,
    IMG_SIZE, save_path_emotion
)

/workspace/yonghak/image_emotion_dataset.npz 저장 완료 (총 8394장)


##### emotion, bounding box crop

In [6]:
# bounding box crop + 감정 저장
def load_labels_and_boxes(label_path):
    labels = {}
    boxes = {}
    for file_name in os.listdir(label_path):
        if file_name.endswith('.json'):
            with open(os.path.join(label_path, file_name), 'r', encoding='euc-kr') as f:
                data = json.load(f)
                for item in data:
                    filename = item['filename']
                    face_exp = item['faceExp_uploader']
                    box_info = item['annot_A']['boxes']
                    bbox = [box_info['minX'], box_info['minY'], box_info['maxX'], box_info['maxY']]
                    labels[filename] = face_exp
                    boxes[filename] = bbox
    return labels, boxes

In [7]:
def prepare_and_save_all_dataset_with_crop(
    train_img_dir, train_labels, train_boxes,
    val_img_dir, val_labels, val_boxes,
    test_img_dir, test_labels, test_boxes,
    img_size, save_path
):
    all_images = []
    all_emotions = []
    all_sets = []

    def add_images(img_dir, labels, boxes, set_name):
        for root, _, files in os.walk(img_dir):
            for file in files:
                if file in labels and file in boxes:
                    img_path = os.path.join(root, file)
                    image = Image.open(img_path).convert('RGB')
                    minX, minY, maxX, maxY = [int(round(x)) for x in boxes[file]]
                    face_img = image.crop((minX, minY, maxX, maxY))
                    face_img = face_img.resize(img_size)
                    all_images.append(np.array(face_img))
                    all_emotions.append(labels[file])
                    all_sets.append(set_name)

    add_images(train_img_dir, train_labels, train_boxes, 'train')
    add_images(val_img_dir, val_labels, val_boxes, 'val')
    add_images(test_img_dir, test_labels, test_boxes, 'test')

    all_images = np.array(all_images)
    all_emotions = np.array(all_emotions)
    all_sets = np.array(all_sets)
    np.savez(save_path, images=all_images, emotions=all_emotions, sets=all_sets)
    print(f"{save_path} 저장 완료 (총 {all_images.shape[0]}장)")

In [8]:
# 실행
save_path_crop = os.path.join(SAVE_DIR, 'image_emotion_crop_dataset.npz')
train_labels, train_boxes = load_labels_and_boxes(TRAIN_LABEL_PATH)
val_labels, val_boxes = load_labels_and_boxes(VALID_LABEL_PATH)
test_labels, test_boxes = load_labels_and_boxes(TEST_LABEL_PATH)

prepare_and_save_all_dataset_with_crop(
    TRAIN_IMG_PATH, train_labels, train_boxes,
    VALID_IMG_PATH, val_labels, val_boxes,
    TEST_IMG_PATH, test_labels, test_boxes,
    IMG_SIZE, save_path_crop
)

/workspace/yonghak/image_emotion_crop_dataset.npz 저장 완료 (총 8394장)


##### AffectNet

In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from PIL import Image

# 경로 설정
pretrain_dir = '/workspace/data/AffectNet_unzipped/affectnet_cleaned'
csv_path = os.path.join(pretrain_dir, 'labels_filtered_cleaned.csv')
save_path = '/workspace/yonghak/vit2_pretrain_emotion_full_dataset.npz'

# CSV에서 라벨 매핑 정보 읽기
df = pd.read_csv(csv_path)
# 문자열 감정 → 정수 인덱스 매핑
mapping = {'happy':0, 'surprise':1, 'anger':2, 'sad':3}
# 상대경로(pth) → 정수 라벨
label_map = {row['pth']: mapping[row['label']] for _, row in df.iterrows()}

# 디스크에 실제 존재하는 경로만 교집합으로 걸러내기
disk_paths = {
    os.path.relpath(os.path.join(r, f), pretrain_dir)
    for r, _, fs in os.walk(pretrain_dir) for f in fs
}
valid_paths = sorted(set(label_map.keys()) & disk_paths)
print(f"사전학습에 사용할 이미지 수: {len(valid_paths)}")

# 이미지 배열과 라벨 배열로 메모리에 로드
images = []
labels = []
for rel in valid_paths:
    img_path = os.path.join(pretrain_dir, rel)
    img = Image.open(img_path).convert('RGB').resize((224,224))
    images.append(np.array(img))
    labels.append(label_map[rel])
images = np.stack(images)    # shape: (N, 224, 224, 3)
labels = np.array(labels)    # shape: (N,)

# 클래스 분포 기반 가중치 계산
counts = Counter(labels)
total  = sum(counts.values())
num_classes = len(mapping)
class_weights = np.array(
    [ total / counts[i] for i in range(num_classes) ],
    dtype=np.float32
)

# NPZ 파일로 저장
np.savez(
    save_path,
    images=images,
    labels=labels,
    class_weights=class_weights
)
print(f"Saved full pretrain dataset to {save_path}")
print(f" images shape      : {images.shape}")
print(f" labels shape      : {labels.shape}")
print(f" class_weights     : {class_weights}")

사전학습에 사용할 이미지 수: 11207
Saved full pretrain dataset to /workspace/yonghak/vit2_pretrain_emotion_full_dataset.npz
 images shape      : (11207, 224, 224, 3)
 labels shape      : (11207,)
 class_weights     : [2.9330018 3.7734008 4.6618137 5.5700793]
